# MongoEngine

## 1. Setup

### 1.1. Imports

In [ ]:
import mongoengine as engine

from mongoengine import (Document, EmbeddedDocument, IntField, LongField, StringField, ObjectIdField,
                         DateField, DateTimeField, ListField, ReferenceField, EmbeddedDocumentField)
from mongoengine.queryset.visitor import Q

from bson.objectid import ObjectId
from datetime import date, datetime as dt

import re

### 1.2. Utils

In [ ]:
def format_array(elements):
    if not elements:
        return '[]'
    
    if len(elements) < 2:
        return '[{}]'.format(elements[0])
    
    output = ['[\n']
    for e in elements:
        output.append('{}{}\n'.format(' ' * 4, e))
    output.append(']')
    return ''.join(output)

## 2. Connect to MongoDB

### 2.1. MongoEngine connection

Arguments:
- `db`: which database will be connected
- `host`: mongo host name or ip address
- `username`: username for connected database
- `password`: password for connected username
- `alias`: name of this connection, for connection more difference hosts

In [ ]:
conn = engine.connect(db='test_db', host='localhost:27017', alias='test_db')
print('* db connectioned, connection is: {}'.format(conn))

### 2.2. Get PyMongo instance

In [ ]:
db = engine.get_db(alias='test_db')
print('* used db is: {}'.format(db))

## 3. Use MongoEngine

### 3.1. Document

#### 3.1.1. Define Document class

`Document` class define a collection, `meta` means metadata of collection
- `db_alias`: which connection should be used
- `collection`: name of collection which should be mapping to

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    id_ = LongField(min_value=0, required=True, db_field='id', unique=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True, default='M')

    def __str__(self):
        return 'User(id={}, id_={}, name="{}", gender="{}")'.format(self.id, self.id_, self.name, self.gender)


try:
    # try to save normal Document object
    user = User(id_=1, name='Alvin', gender='M')
    user.save()
    print('* user {} was created'.format(user))

    # save Document with default field gender
    user = User(id_=2, name='Authur')
    user.save()
    print('\n* user {} was created'.format(user))

    # list all saved objects
    users = User.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # try to trigger key duplicate error (by id_ field duplicated)
    try:
        user = User(id_=2, name='Authur')
        user.save()
    except Exception as err:
        print('\n* error caused: {}'.format(err))

    # try to trigger validation error (by gender not match regex rule)
    try:
        user = User(id_=3, name='Emma', gender='X')
        user.save()
    except Exception as err:
        print('\n* error caused: {}'.format(err))
finally:
    db.drop_collection('user')

#### 3.1.2. Id of Document

- `primary_key`: physics id of collection (_id property of collection)
- `id`: readonly value mappting to primary key (cannot define `id` property for Document explicit)
- `ObjectIdField`: object id type, required and primary key by default

##### 3.1.2.1. `ObjectIdField`

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    key = ObjectIdField()  # key field, required and primary key by default
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        # self.id mappting to primary key (key field)
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)


try:
    # ignore key property, use default ObjectId value

    for n in range(1, 4):
        user = User(name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # use ObjectId object as value of key

    for n in range(1, 4):
        user = User(key=ObjectId(), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # use user define ObjectId as value of key

    for n in range(1, 4):
        user = User(key=ObjectId('{}'.format(n) * 24), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

##### 3.1.2.2. `primary_key`

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    key = StringField(max_length=10, min_length=2, required=True, primary_key=True)  # primary key
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        # self.id mappting to primary key (key field)
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)


try:
    for n in range(1, 4):
        user = User(key='a' * (n + 2), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

##### 3.1.2.3. `_id` field and primary key

- Demo 1

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    # _id field means _id property of collection
    # if without 'primary_key', '_id' field cannot mapping to 'id' field
    _id = StringField(required=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}")'.format(self._id, self.name, self.gender)


try:
    user = User(_id='user_1', name='Alvin1', gender='M' if n % 2 == 0 else 'F')
    user.save()

    user = User(_id='user_1', name='Alvin2', gender='M' if n % 2 == 0 else 'F')
    user.save()  # save document with same _id, means update collection by same _id

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

- Demo 2

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    # if with 'primary_key', '_id' field can mapping to 'id' field
    _id = StringField(required=True, primary_key=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)


try:
    user = User(_id='user_1', name='Alvin1', gender='M' if n % 2 == 0 else 'F')
    user.save()

    user = User(_id='user_1', name='Alvin2', gender='M' if n % 2 == 0 else 'F')
    user.save()  # save document with same primary key, means update collection by same primary key

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

#### 3.1.3. Date & Datetime field

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    birthday = DateField(required=True)
    created_at = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}", birthday={}, created_at={})'.format(
            self.id, self.name, self.gender, self.birthday, self.created_at)


try:
    user = User(name='Alvin', gender='M', birthday=date(1981, 3, 17))
    user.save()

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

#### 3.1.4. ListField field

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    hobbies = ListField(StringField(required=True), required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}", hobbies={})'.format(self.id, self.name, self.gender, self.hobbies)


try:
    user = User(name='Alvin', gender='M', hobbies=['Football', 'Gaming'])
    user.save()

    users = User.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # try to trigger validation error (list field include empty element)
    try:
        user = User(name='Alvin', gender='M', hobbies=[])
        user.save()
    except Exception as err:
        print('* error caused: {}'.format(err))
finally:
    db.drop_collection('user')

#### 3.1.5. Reference field

In [ ]:
# Category document
class Category(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'category'
    }

    name = StringField(max_length=30, required=True)
    artnum = IntField(default=0, required=True)
    date = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'Category(id={}, name="{}", artnum={}, date={})'.format(self.id, self.name, self.artnum, self.date)

# Post document
class Post(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'post'
    }

    title = StringField(max_length=100, required=True)
    content = StringField(required=True)
    tags = ListField(StringField(max_length=20, required=True), required=True)
    category = ReferenceField(Category)

    def __str__(self):
        return 'Post(id={}, title="{}", content={}, tags={}, \n\t category={})'.format(self.id, self.title, self.content,
                                                                                       self.tags, self.category)


try:
    # create some categories
    category1 = Category(name='C1', artnum=1)
    category1.save()

    category2 = Category(name='C2', artnum=2)
    category2.save()

    categories = Category.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(categories), format_array(categories)))

    # create some posts, and each post reference category
    post = Post(title='POST 1', content='This is a first post', tags=['t1', 't2'], category=category1)
    post.save()

    post = Post(title='POST 2', content='This is a second post', tags=['t2', 't3'], category=category2)
    post.save()

    posts = Post.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(posts), format_array(posts)))
finally:
    db.drop_collection('category')
    db.drop_collection('post')

### 3.2. EmbeddedDocument

In [ ]:
# Category document
class Category(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'category'
    }

    name = StringField(max_length=30, required=True)
    artnum = IntField(default=0, required=True)
    date = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'Category(id={}, name="{}", artnum={}, date={})'.format(self.id, self.name, self.artnum, self.date)

# Tag document
class Tag(EmbeddedDocument):
    name = StringField(max_length=100, required=True)
    created_at = DateTimeField(default=dt.utcnow())

    def __str__(self):
        return 'Tag(name="{}", created_at={})'.format(self.name, self.created_at)

# Post document
class Post(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'post'
    }

    title = StringField(max_length=100, required=True)
    content = StringField(required=True)
    category = ReferenceField(Category, required=True)
    tags = ListField(EmbeddedDocumentField(Tag))

    def __str__(self):
        return 'Post(id={}, title="{}", content={}, tags={}, \n\t category={})'.format(self.id, self.title, self.content,
                                                                                       self.tags, self.category)


try:
    # create some categories
    category1 = Category(name='C1', artnum=1)
    category1.save()

    category2 = Category(name='C2', artnum=2)
    category2.save()

    categories = Category.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(categories), format_array(categories)))

    # create some posts, and each post reference category
    post = Post(title='POST 1', content='This is a first post', category=category1)
    post.tags.append(Tag(name='t1'))
    post.save()

    # create tags
    post.tags.append(Tag(name='t2'))
    post.save()

    posts = Post.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(posts), format_array(posts)))
finally:
    db.drop_collection('category')
    db.drop_collection('post')

### 3.3. Query

#### 3.3.1. Simple query

compare filter operators:
- `ne`: not equal to
- `lt`: less than
- `lte`: less than or equal to
- `gt`: greater than
- `gte`: greater than or equal to
- `not`: negate a standard check, may be used before other operators (e.g. Q(age__not__mod=(5, 0)))
- `in`: value is in list (a list of values should be provided)
- `nin`: value is not in list (a list of values should be provided)
- `mod`: value % x == y, where x and y are two provided values
- `all`: every item in list of values provided is in array
- `size`: the size of the array is
- `exists`: value for field exists

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    no = IntField(min=1, required=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    birthday = DateField(required=True)
    created_at = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'User(no={}, name="{}", gender="{}", birthday={}, created_at={})'.format(
            self.no, self.name, self.gender, self.birthday, self.created_at)


try:
    User(no=1, name='Alvin', gender='M', birthday=date(1981, 3, 17)).save()
    User(no=2, name='Emma', gender='F', birthday=date(1985, 3, 29)).save()
    User(no=3, name='Lucy', gender='F', birthday=date(1988, 2, 9)).save()

    users = User.objects
    print('* all objects in db are: {}'.format(format_array(users)))

    users = User.objects.all()
    print('\n* all objects in db are: {}'.format(format_array(users)))

    users = User.objects(name='Alvin')
    print('\n* found objects in db are: {}'.format(format_array(users)))
    
    users = User.objects(name__ne='Alvin')
    print('\n* found objects in db are: {}'.format(format_array(users)))
    
    users = User.objects(no__gt=1)
    print('\n* found objects in db are: {}'.format(format_array(users)))
    
    users = User.objects(birthday__gt=date(1981, 3, 1), birthday__lt=date(1985, 3, 31))
    print('\n* found objects in db are: {}'.format(format_array(users)))
finally:
    db.drop_collection('user')

#### 3.3.2. String query

string field filter operators:
- `exact`: string field exactly matches value
- `iexact`: string field exactly matches value (case insensitive)
- `contains`: string field contains value
- `icontains`: string field contains value (case insensitive)
- `startswith`: string field starts with value
- `istartswith`: string field starts with value (case insensitive)
- `endswith`: string field ends with value
- `iendswith`: string field ends with value (case insensitive)

match string field by regex:

```python
import re

cond = re.compile(r'<regex express>')
results = Collections.objects(<field>__regex=cond)
```

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    no = IntField(min=1, required=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    birthday = DateField(required=True)
    created_at = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'User(no={}, name="{}", gender="{}", birthday={}, created_at={})'.format(
            self.no, self.name, self.gender, self.birthday, self.created_at)


try:
    User(no=1, name='Alvin', gender='M', birthday=date(1981, 3, 17)).save()
    User(no=2, name='Emma', gender='F', birthday=date(1985, 3, 29)).save()
    User(no=3, name='Lucy', gender='F', birthday=date(1988, 2, 9)).save()

    users = User.objects(name__istartswith='A')
    print('* found objects in db are: {}'.format(format_array(users)))
    
    users = User.objects(name__contains='ma')
    print('\n* found objects in db are: {}'.format(format_array(users)))
    
    # search by regex
    name_regex = re.compile(r'^[A|L]\w+')
    users = User.objects(name=name_regex)
    print('\n* found objects in db are: {}'.format(format_array(users)))
finally:
    db.drop_collection('user')

#### 3.3.3. Combine conditions with 'and', 'or'

In [ ]:
class User(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    no = IntField(min=1, required=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    birthday = DateField(required=True)
    created_at = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'User(no={}, name="{}", gender="{}", birthday={}, created_at={})'.format(
            self.no, self.name, self.gender, self.birthday, self.created_at)


try:
    User(no=1, name='Alvin', gender='M', birthday=date(1981, 3, 17)).save()
    User(no=2, name='Emma', gender='F', birthday=date(1985, 3, 29)).save()
    User(no=3, name='Lucy', gender='F', birthday=date(1988, 2, 9)).save()

    users = User.objects(Q(name__istartswith='A'))
    print('* found objects in db are: {}'.format(format_array(users)))
    
    name_regex = re.compile(r'^[A|L]\w+')
    users = User.objects(Q(name=name_regex) & Q(gender='F'))
    print('\n* found objects in db are: {}'.format(format_array(users)))
    
    users = User.objects(Q(name='Lucy') | Q(gender='M'))
    print('\n* found objects in db are: {}'.format(format_array(users)))
finally:
    db.drop_collection('user')

#### 3.3.4. Query in embedded document

In [ ]:
class Tag(EmbeddedDocument):
    no = IntField(min_value=0, required=True)
    name = StringField(max_length=100, required=True)

    def __str__(self):
        return 'Tag(no={}, name="{}")'.format(self.no, self.name)

class Post(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'post'
    }

    title = StringField(max_length=100, required=True)
    content = StringField(required=True)
    tags = ListField(EmbeddedDocumentField(Tag), required=True)

    def __str__(self):
        return 'Post(title="{}", content={}, tags={})'.format(self.title, self.content, self.tags)


try:
    Post(title='Post 1', content='This is post 1', tags=[Tag(no=1, name='t1'), Tag(no=2, name='t2')]).save()
    Post(title='Post 2', content='This is post 2', tags=[Tag(no=2, name='t2'), Tag(no=3, name='t3')]).save()
    Post(title='Post 3', content='This is post 3', tags=[Tag(no=3, name='t3'), Tag(no=4, name='t4')]).save()
    
    # query by embedded document field
    # the raw query statement: db.post.find({'tags.name': 't1', 'tags.no': 2})
    # results contains both match condition that 'tags.name=t1' or 'tags.no=2', two results could be found
    posts = Post.objects(tags__name='t1', tags__no=2)
    print('* found objects in db are: {}'.format(format_array(posts)))

    # query by embedded document field
    # the raw query statement: db.post.find({'tags': {'$elemMatch': {'name': 't1', 'no': 2}}})
    # nothing should be found
    posts = Post.objects(tags__match={'name': 't1', 'no': 2})
    print('\n* found objects in db are: {}'.format(format_array(posts)))
    
    # in query
    posts = Post.objects(tags__name__in=['t1', 't4'])
    print('\n* found objects in db are: {}'.format(format_array(posts)))
finally:
    db.drop_collection('post')

In [ ]:
engine.disconnect(alias='test_db')